In [6]:
import pandas as pd
import networkx as nx
from collections import defaultdict

# Load the data from the CSV file
file_path = 'preps.csv'
Preps = pd.read_csv(file_path)

# Create a directed graph from the CSV data
G = nx.DiGraph()

# Add edges to the graph based on 'prep' and 'ingredient'
for _, row in Preps.iterrows():
    prep = row['item_descrip']
    ingredient = row['item_descrip.1']
    G.add_edge(prep, ingredient)

# Detect cycles to ensure there are no cyclic dependencies
try:
    cycles = list(nx.find_cycle(G, orientation='original'))
    if cycles:
        print(f"Cycle detected: {cycles}")
        raise ValueError("Cycles detected in prep dependencies!")
except nx.exception.NetworkXNoCycle:
    print("No cycles detected, proceeding with tree creation.")

# Function to recursively build the tree
def build_tree(node, graph):
    tree = {node: []}
    for neighbor in graph.successors(node):
        tree[node].append(build_tree(neighbor, graph))
    return tree

# Generate the hierarchical tree for each prep with no incoming edges (root nodes)
trees = {}
for prep in G.nodes:
    if G.in_degree(prep) == 0:  # Root nodes
        trees[prep] = build_tree(prep, G)

# Function to pretty-print the tree
def pretty_print_tree(tree, indent='', last='updown'):
    """Recursively prints the tree structure in a pretty way."""
    for key, value in tree.items():
        if last == 'updown':  # If at the top of the tree
            pointers = {'updown': '─┬─', 'mid': '├─', 'end': '└─'}
        else:
            pointers = {'updown': ' ', 'mid': '├─', 'end': '└─'}

        print(indent + pointers[last] + key)
        if isinstance(value, list):
            for idx, item in enumerate(value):
                if isinstance(item, dict):
                    last_key = 'end' if idx == len(value) - 1 else 'mid'
                    pretty_print_tree(item, indent + ('│  ' if last == 'mid' else '   '), last_key)

# Convert nested defaultdict to a regular dict for better readability
def convert_tree(d):
    if isinstance(d, defaultdict):
        d = {k: convert_tree(v) for k, v in d.items()}
    elif isinstance(d, list):
        d = [convert_tree(v) for v in d]
    return d

# Convert trees into readable format
trees = {k: convert_tree(v) for k, v in trees.items()}

# Output the hierarchical trees for each prep
for prep, tree in trees.items():
    print(f"Tree for {prep}:")
    pretty_print_tree(tree)
    print()  # Add space between trees for readability


No cycles detected, proceeding with tree creation.
Tree for To Go Cutlery 2023:
─┬─To Go Cutlery 2023
   ├─CUP PORT PAPER WHT 2Z
   ├─LID Foil Board 6x8.5 SO
   ├─Napkin Bev 2ply White
   ├─CUTLERY FORK WOOD bulk
   ├─CUTLERY SPOON WOOD soup
   └─Foil pan 6x8.5 oblong 2.25lb

Tree for 2023 Beets Salad Prep:
─┬─2023 Beets Salad Prep
   ├─ORANGES 5lb 
   ├─ARUGULA WILD BABY
   ├─GOAT CHS CRUMBLE 
   ├─Beets Prep 2023
   │  ├─BEETS
   │  ├─THYME
   │  ├─SALT KOSHER COARSE
   │  ├─SPICE PEPPER BLACK FINE 540G
   │  └─PFO OIL CANOLA JIB  ANTI FOAM
   ├─Candy Cane beets Prep 2023
   │  ├─THYME
   │  ├─SALT KOSHER COARSE
   │  ├─SPICE PEPPER BLACK FINE 540G
   │  ├─PFO OIL CANOLA JIB  ANTI FOAM
   │  └─JIT-BEET Candy Cane Teen 5lb
   ├─Golden Beets Prep 2023
   │  ├─THYME
   │  ├─SALT KOSHER COARSE
   │  ├─SPICE PEPPER BLACK FINE 540G
   │  ├─PFO OIL CANOLA JIB  ANTI FOAM
   │  └─Beets gold 5lb
   ├─2023 Candied walnut
   │  ├─WALNUT PIECES 1/2 & pcs
   │  ├─Water - Tap
   │  ├─SYRUP CORN GOL